<a href="https://colab.research.google.com/github/limshaocong/analyticsEdge/blob/main/FAANG_Volume_Prediction_caa%2024Nov21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preliminaries**

In [5]:
suppressMessages(library(tidyverse)) # generic must have package
suppressMessages(library(dplyr))
suppressMessages(library(ggplot2)) # plotting package
suppressMessages(library(lubridate)) # easy comprehension of dates from string to correct datetime format
suppressMessages(library(data.table))
suppressMessages(library(purrr)) # reduce
if("patchwork" %in% rownames(installed.packages()) == FALSE) {install.packages("patchwork")}
suppressMessages(library(patchwork))
if("caret" %in% rownames(installed.packages()) == FALSE) {install.packages("caret")}
suppressMessages(library(caret))

options(repr.plot.width = 10,
        repr.plot.height = 9,
        repr.plot.pointsize = 20)

Import data and check for any NA within the file

In [8]:
path = "https://raw.githubusercontent.com/limshaocong/analyticsEdge/main/Datasets/FAANG/altdata.csv"
df = read.csv(path) %>% mutate(date = ymd(date)) %>% select(- open, - close, - high, - low)

if (dim(df)[1] == dim(na.omit(df))[1]) {
  print("No missing data.")
} else {
  print("Missing data")
}

[1] "No missing data."


# **Exploratory Analysis**

Overview of Data

In [9]:
head(df, 5)

,date,ticker,vol,newssentiment,newsmentions,twtrmentions,twtrsentiment,wsbsentiment,wsbmentions,retailvol,⋯,twtrmentions5MA,wsbmentions5MA,newsmentions5MA,twtrmentions10MA,wsbmentions10MA,newsmentions10MA,retailvollag1,retailvollag2,retailvollag4,target
,<date>,<chr>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>
1,2019-01-08,FB,26252863,63.0,2,569,0.26889279,0.3528667,1,11163579,⋯,473.6,1,0.8,487.8,1,0.6,11163579,10410940,8964350,9658300
2,2019-01-09,FB,22203279,50.0,0,489,0.09406953,-0.3818000,1,9658300,⋯,461.0,1,0.8,473.5,1,0.4,9658300,8218650,8249882,6779001
3,2019-01-10,FB,16111304,50.5,2,464,0.17456897,0.3246000,1,6779001,⋯,457.8,1,0.8,469.0,1,0.5,6779001,6088824,7751706,5398647
4,2019-01-11,FB,12907031,50.0,0,368,0.17663043,-0.0799400,1,5398647,⋯,467.2,1,0.4,487.2,1,0.6,5398647,7284762,7707233,9170878
5,2019-01-14,FB,20515678,50.0,0,399,0.23809524,-0.1558800,1,9170878,⋯,462.0,1,0.4,495.3,1,0.6,9170878,9325642,7777200,9480406


In [10]:
str(df)

'data.frame':	3310 obs. of  40 variables:
 $ date             : Date, format: "2019-01-08" "2019-01-09" ...
 $ ticker           : chr  "FB" "FB" "FB" "FB" ...
 $ vol              : int  26252863 22203279 16111304 12907031 20515678 24065513 18060414 15787914 32309412 22393694 ...
 $ newssentiment    : num  63 50 50.5 50 50 50 50 37 40 50 ...
 $ newsmentions     : int  2 0 2 0 0 0 0 1 1 0 ...
 $ twtrmentions     : int  569 489 464 368 399 616 463 476 613 496 ...
 $ twtrsentiment    : num  0.2689 0.0941 0.1746 0.1766 0.2381 ...
 $ wsbsentiment     : num  0.3529 -0.3818 0.3246 -0.0799 -0.1559 ...
 $ wsbmentions      : int  1 1 1 1 1 1 1 1 1 1 ...
 $ retailvol        : int  11163579 9658300 6779001 5398647 9170878 9480406 7058870 5805217 11946377 8903335 ...
 $ instvol          : int  15100214 12547579 9346003 7509384 11349400 14588416 10966844 9982697 19083235 13475359 ...
 $ retailperc       : num  0.425 0.435 0.42 0.418 0.447 ...
 $ newssentimentlag1: num  50 63 50 50.5 50 50 50 50 37 40

662 trading days worth of training data from Jan 8, 2019 to Aug 30, 2021.

In [11]:
df %>%
  group_by(ticker) %>%
  summarise(n())

ticker,n()
<chr>,<int>
AAPL,662
AMZN,662
FB,662
GOOGL,662
NFLX,662


# **Data Preparation**

Train-test split

In [25]:
split = as.Date("2020-11-24")

# Train-test split
train = df %>% filter(date < split)
test = df %>% filter(date >= split)

train_days = dim(train)[1]/5
test_days = dim(test)[1]/5

train_prop = train_days / (train_days + test_days)

paste0("Training data proportion: ", round(train_prop * 100, 1), "%. Total training days = ", train_days)

[1] "Training data proportion: 71.8%. Total training days = 475"

Train-validate Split - as normal k-fold CV does not work on time series, an expanding window approach is used (see Section 4.3 of https://topepo.github.io/caret/data-splitting.html#time). With these parameters, CV-error will be run on 5 different validation sets.

In [26]:
index = 1:train_days
slices = createTimeSlices(index, initialWindow = 230, horizon = 100, fixedWindow = FALSE, skip = 28)

trainslices = slices[[1]] # specific slices callable by df[trainslices[[i]],]
testslices = slices[[2]]

lapply(slices, length)

$train
[1] 6

$test
[1] 6

In [28]:
for (fold in 1:6) {
  trainN = length(slices$train[[fold]])
  testN = length(slices$test[[fold]])
  trainperc = round(trainN / (trainN + testN), 2) * 100

  print(paste0("Fold ", fold, ": ", trainN, " train data. 100 test data. ", trainperc, "%"))
}

[1] "Fold 1: 230 train data. 100 test data. 70%"
[1] "Fold 2: 259 train data. 100 test data. 72%"
[1] "Fold 3: 288 train data. 100 test data. 74%"
[1] "Fold 4: 317 train data. 100 test data. 76%"
[1] "Fold 5: 346 train data. 100 test data. 78%"
[1] "Fold 6: 375 train data. 100 test data. 79%"


In [29]:
slices$train
slices$test

$Training230
  [1]   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
 [19]  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
 [37]  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
 [55]  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
 [73]  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
 [91]  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
[109] 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
[127] 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
[145] 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
[163] 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
[181] 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
[199] 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
[217] 217 218 219 220 221 222 223 224 225 226 227 228 229 230

$Training259
  [1]   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
 [19]  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
 [37]  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
 [55]  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
 [73]  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
 [91]  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
[109] 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
[127] 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
[145] 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
[163] 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
[181] 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
[199] 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
[217] 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
[235] 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252
[253] 253 254 255 256 257 258 259

$Training288
  [1]   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
 [19]  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
 [37]  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
 [55]  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
 [73]  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
 [91]  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
[109] 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
[127] 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
[145] 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
[163] 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
[181] 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
[199] 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
[217] 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
[235] 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252
[253] 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270
[271] 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288

$Training317
  [1]   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
 [19]  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
 [37]  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
 [55]  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
 [73]  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
 [91]  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
[109] 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
[127] 127 128 129 130 131 1

$Testing230
  [1] 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248
 [19] 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266
 [37] 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284
 [55] 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302
 [73] 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320
 [91] 321 322 323 324 325 326 327 328 329 330

$Testing259
  [1] 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277
 [19] 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295
 [37] 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313
 [55] 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331
 [73] 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349
 [91] 350 351 352 353 354 355 356 357 358 359

$Testing288
  [1] 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306
 [19] 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324
 [37] 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342
 [55] 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360
 [73] 361 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378
 [91] 379 380 381 382 383 384 385 386 387 388

$Testing317
  [1] 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335
 [19] 336 337 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353
 [37] 354 355 356 357 358 359 360 361 362 363 364 365 366 367 368 369 370 371
 [55] 372 373 374 375 376 377 378 379 380 381 382 383 384 385 386 387 388 389
 [73] 390 391 392 393 394 395 396 397 398 399 400 401 402 403 404 405 406 407
 [91] 408 409 410 411 412 413 414 415 416 417

$Testing346
  [1] 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364
 [19] 365 366 367 368 369 370 371 372 373 374 375 376 377 378 379 380 381 382
 [37] 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399 400
 [55] 401 402 403 404 405 406 407 408 409 410 411 412 413 414 415 416 417 418
 [73] 419 420 421 422 423 424 425 426 427 428 429 430 431 432 433 434 435 436
 [91] 437 438 439 440 441 442 443 444 445 446

$Testing375
  [1] 376 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393
 [19] 394 395 396 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411
 [37] 412 413 414 415 416 417 418 419 420 421 422 423 424 425 426 427 428 429
 [55] 430 431 432 433 434 435 436 437 438 439 440 441 442 443 444 445 446 447
 [73] 448 449 450 451 452 453 454 455 456 457 458 459 460 461 462 463 464 465
 [91] 466 467 468 469 470 471 472 473 474 475

# **Model Building**

## **Defining functions for various models**

In [45]:
if("Metrics" %in% rownames(installed.packages()) == FALSE) {install.packages("Metrics")}
suppressMessages(library(Metrics))
if("randomForest" %in% rownames(installed.packages()) == FALSE) {install.packages("randomForest")}
suppressMessages(library(randomForest))
if("xgboost" %in% rownames(installed.packages()) == FALSE) {install.packages("xgboost")}
library(xgboost)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [31]:
install.packages("zoo")
library(zoo) # rolling mean

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric




In [ ]:
# Naive model using 5day MA

naive = df %>%
  group_by(ticker) %>%
  mutate(vol5MA = rollmean(vol, align = "right", k = 5, fill = NA)) %>%
  select(date, ticker, voltarget, vol5MA)

#naive

applCVrmse = 0
applCVmae = 0

for (i in 1:7) {
  validation = naive[testslices[[i]],]
  rmse = rmse(validation$voltarget, validation$vol5MA)
  mae = mae(validation$voltarget, validation$vol5MA)

  applCVrmse[i] = rmse
  applCVmae[i] = mae

}

applCVmae

[1] 4756497 6142279 7030798 8089921 8389126 8204951 7064016

In [236]:
# Function to construct CART tree
cart = function(trainX, trainY) {
  
  train_control = trainControl(method = "timeslice",
                            initialWindow = 130,
                            horizon = 30,
                            fixedWindow = FALSE,
                            skip = 40,
                            savePredictions = TRUE)

  # hyperparameters
  cp_values = data.frame(.cp = seq(0, 0.02, by = 0.0001))

  model = train(x = trainX,
                y = trainY,
                method = "rpart",
                trControl = train_control,
                tuneGrid = cp_values)

}

# Function to construct RandomForest
randomforest = function(trainX, trainY) {

  train_control = trainControl(method = "timeslice",
                                initialWindow = 130,
                                horizon = 30,
                                fixedWindow = FALSE,
                                skip = 40,
                                savePredictions = TRUE)

  # hyperparameters
  n_pred = dim(trainX)[2]
  mtry_low = round(0.4 * n_pred)
  mtry_upp = round(0.6 * n_pred)
  mtry_grid = data.frame(mtry = seq(mtry_low, mtry_upp, by = 1))
      
  model = train(x = trainX,
                y = trainY,
                method = "rf",
                trControl = train_control,
                tuneGrid = mtry_grid,
                ntree = 300,
                nodesize = 5)

}

# Function to construct XGBoost
xgb = function(trainX, trainY) {
  
  train_control = trainControl(method = "timeslice",
                              initialWindow = 130,
                              horizon = 30,
                              fixedWindow = FALSE,
                              skip = 40,
                              savePredictions = TRUE,
                              allowParallel = TRUE)

  # hyperparameters that require further tuning
  parm_grid = expand.grid(nrounds = 100, 
                          max_depth = 6, 
                          eta = seq(0.01, 0.05, by = 0.001),
                          gamma = 0,
                          colsample_bytree = 1,
                          min_child_weight = 1,
                          subsample = c(0.3, 0.5))

  model = train(x = trainX,
                y = trainY,
                method = "xgbTree",
                tuneGrid = parm_grid,
                trControl = train_control)

}

# Function to plot 
plotlastfold = function(model) {
  
  train_pred = predict(model$finalModel, newdata = trainX)
  
  model_df = data.frame(timesteps = seq(1, train_days), predicted = train_pred, actual = trainY)

  ggplot(data = model_df, (aes(x = timesteps))) +
    geom_line(aes(y = predicted), color = "blue") +
    geom_line(aes(y = actual), color = "black")

}


In [202]:
# $ date             : Date, format: "2019-01-08" "2019-01-09" ...
# $ ticker           : chr  "FB" "FB" "FB" "FB" ...
# $ vol              : int  26252863 22203279 16111304 12907031 20515678 24065513 18060414 15787914 32309412 22393694 ...
# $ newssentiment    : num  63 50 50.5 50 50 50 50 37 40 50 ...
# $ newsmentions     : int  2 0 2 0 0 0 0 1 1 0 ...
# $ twtrmentions     : int  569 489 464 368 399 616 463 476 613 496 ...
# $ twtrsentiment    : num  0.2689 0.0941 0.1746 0.1766 0.2381 ...
# $ wsbsentiment     : num  0.3529 -0.3818 0.3246 -0.0799 -0.1559 ...
# $ wsbmentions      : int  1 1 1 1 1 1 1 1 1 1 ...
# $ retailvol        : int  11163579 9658300 6779001 5398647 9170878 9480406 7058870 5805217 11946377 8903335 ...
# $ instvol          : int  15100214 12547579 9346003 7509384 11349400 14588416 10966844 9982697 19083235 13475359 ...
# $ retailperc       : num  0.425 0.435 0.42 0.418 0.447 ...
# $ newssentimentlag1: num  50 63 50 50.5 50 50 50 50 37 40 ...
# $ newsmentionslag1 : int  0 2 0 2 0 0 0 0 1 1 ...
# $ twtrmentionslag1 : int  415 569 489 464 368 399 616 463 476 613 ...
# $ twtrsentimentlag1: num  0.2024 0.2689 0.0941 0.1746 0.1766 ...
# $ wsbsentimentlag1 : num  -0.2222 0.3529 -0.3818 0.3246 -0.0799 ...
# $ wsbmentionslag1  : int  1 1 1 1 1 1 1 1 1 1 ...
# $ newssentimentlag2: num  50 50 63 50 50.5 50 50 50 50 37 ...
# $ newsmentionslag2 : int  0 0 2 0 2 0 0 0 0 1 ...
# $ twtrmentionslag2 : int  431 415 569 489 464 368 399 616 463 476 ...
# $ twtrsentimentlag2: num  0.058 0.2024 0.2689 0.0941 0.1746 ...
# $ wsbsentimentlag2 : num  -0.0623 -0.2222 0.3529 -0.3818 0.3246 ...
# $ wsbmentionslag2  : int  1 1 1 1 1 1 1 1 1 1 ...
# $ newssentimentlag4: num  50 50 50 50 63 50 50.5 50 50 50 ...
# $ newsmentionslag4 : int  2 0 0 0 2 0 2 0 0 0 ...
# $ twtrmentionslag4 : int  606 521 431 415 569 489 464 368 399 616 ...
# $ twtrsentimentlag4: num  0.1172 0.0307 0.058 0.2024 0.2689 ...
# $ wsbsentimentlag4 : num  0.1815 -0.1377 -0.0623 -0.2222 0.3529 ...
# $ wsbmentionslag4  : int  1 1 1 1 1 1 1 1 1 1 ...
# $ twtrmentions5MA  : num  474 461 458 467 462 ...
# $ wsbmentions5MA   : num  1 1 1 1 1 1 1 1 1 1 ...
# $ newsmentions5MA  : num  0.8 0.8 0.8 0.4 0.4 0.2 0.4 0.4 0.6 1.2 ...
# $ twtrmentions10MA : num  488 474 469 487 495 ...
# $ wsbmentions10MA  : num  1 1 1 1 1 1 1 1 1 1 ...
# $ newsmentions10MA : num  0.6 0.4 0.5 0.6 0.6 0.5 0.8 0.6 0.8 1.1 ...
# $ retailvollag1    : int  11163579 9658300 6779001 5398647 9170878 9480406 7058870 5805217 11946377 8903335 ...
# $ retailvollag2    : num  10410940 8218650 6088824 7284762 9325642 ...
# $ retailvollag4    : num  8964350 8249882 7751706 7707233 7777200 ...
# $ target           : int  9658300 6779001 5398647 9170878 9480406 7058870 5805217 11946377 8903335 7344744 ...

full_feature = c(3:39)
reduced_feature = c(3:12)

In [252]:
run_models = function(x, feature_set) {

  output = list() # initialize list to store results
  output[["ticker"]] = x # add in ticker

  # Create trainX, trainY, testX and testY based on feature_set
  trainX = train %>% filter(ticker == x) %>% select(feature_set)
  trainY = train %>% filter(ticker == x) %>% select(c(40)) %>% pull()
  testX = test %>% filter(ticker == x) %>% select(feature_set)
  testY = test %>% filter(ticker == x) %>% select(c(40)) %>% pull()

  # Run CART Model
  # Extract and record error metrics
  tree = cart(trainX, trainY)
  output[["dtparams"]] = as.list(tree$bestTune)
  output[["dtRMSE"]] = tree$results %>% arrange(RMSE) %>% dplyr::slice(1) %>% select(RMSE) %>% as.double(.)
  output[["dtRMSESD"]] = tree$results %>% arrange(RMSE) %>% dplyr::slice(1) %>% select(RMSESD) %>% as.double(.)
  output[["dtMAE"]] = tree$results %>% arrange(RMSE) %>% dplyr::slice(1) %>% select(MAE) %>% as.double(.)
  output[["dtMAESD"]] = tree$results %>% arrange(RMSE) %>% dplyr::slice(1) %>% select(MAESD) %>% as.double(.)
  
  # Run RF Model
  # Extract and record error metrics
  set.seed(15071)
  forest = randomforest(trainX, trainY)
  output[["rfparams"]] = as.list(forest$bestTune) 
  output[["rfRMSE"]] = forest$results %>% arrange(RMSE) %>% dplyr::slice(1) %>% select(RMSE) %>% as.double(.)
  output[["rfRMSESD"]] = forest$results %>% arrange(RMSE) %>% dplyr::slice(1) %>% select(RMSESD) %>% as.double(.)
  output[["rfMAE"]] = forest$results %>% arrange(RMSE) %>% dplyr::slice(1) %>% select(MAE) %>% as.double(.)
  output[["rfMAESD"]] = forest$results %>% arrange(RMSE) %>% dplyr::slice(1) %>% select(MAESD) %>% as.double(.)

  output = output

}

In [253]:
fb = run_models("FB", reduced_feature)
#aapl = run_models("AAPL", reduced_feature)
#amzn = run_models("AMZN", reduced_feature)
#nflx = run_models("NFLX", reduced_feature)
#googl = run_models("GOOGL", reduced_feature)

In [ ]:
forestImp = as.data.frame(rbind(forest$finalModel$importance)) %>% arrange(desc(IncNodePurity))
head(forestImp, 10)

In [111]:
# Construct a XGBoost
xgboost = xgb()

#xgboost$bestTune

# The above parameters gives the lowest CV RMSE
# xgBoost requires far more tuning
# The current tuning grid is too small to explore the solution space fully
xgboost$results %>% arrange(RMSE) %>% head()